# 基于计数的方法
基于计数的方法的目标是从富有实践知识的语料库中，自动且高效地提取本质。
- 语料库 corpus
    大量的文本数据。语料库中包含了大量的关于自然语言的实践知识，即文章的写作方法、单词的选择方法和单词含义等。
    > 自然语言处理领域中使用的语料库有时会给文本数据添加额外的信息。比如，可以给文本数据的各个单词标记词性。在这种情况下，为了方便计算机处理，语料库通常会被结构化（比如，采用树结构等数据形式）。

## Python 预处理
单纯的字符串处理

In [ ]:
import numpy as np


def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            word_id = len(word_to_id)
            word_to_id[word] = word_id
            id_to_word[word_id] = word
    corpus = np.array([word_to_id[w] for w in words])
    return corpus, word_to_id, id_to_word


text = 'You say goodbye and I say hello.'
print(preprocess(text))

## 单词的分布式表示
将单词表示为固定长度的向量，准确把握单词含义。
多为密集向量（多数元素是非0实数）

分布式表示的前提是 **分布式假设**， 即某个单词的含义由它周围的词形成。 许多用向量表示单词的研究多是基于此假设。
![2_3_1](images/2_3_1.png)
**上下文**：某个单词的周围词汇。上下位的大小 成为 **窗口大小**，

## 共现矩阵 co-occurrence matrix
基于计数(或统计)的方法：针对某个词，对其周围出现了多少次什么单词进行计数，再汇总。
如图，每一行的值表示，针对行左侧的词，在其**上下文**中每列对应词出现的次数, 窗口大小为 1:

![2_3_2](images/2_3_2.png)

以矩阵的形式，表示图中数据，即为共现矩阵。

In [ ]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    """ 创建共现矩阵 """
    corpus_size = len(corpus)
    # 以 0 初始化一个 vocab_size x vocab_size 的矩阵
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    # 遍历语料库 corpus 中的每一个 word id
    for idx, word_id in enumerate(corpus):
        # 根据窗口大小遍历上下文的所有单词
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id][left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id][right_word_id] += 1

    return co_matrix


corpus, word_to_id, id_to_word = preprocess(text)
C = create_co_matrix(corpus, len(word_to_id))
print(C)

## 向量相似度 vector similarity
测量 向量相似度的方法有:
- 向量内积:两个形状相同的向量，对应位置元素一一相乘后再求和，是一个标量，又称为标量积、点积
- 欧氏距离（欧几里得度量）:指在m维空间中两个点之间的真实距离，或者向量的自然长度（即该点到原点的距离）。 
等...

但是测量单词的向量表示的相似度，余弦相似度（cosine similarity）比较常用,定义如下：
![2_3_3](images/2_3_3.png)
分子：向量内积

分母：各个向量的L2范数

范数：一个n维度的欧几里得空间有欧氏范数,每一个向量的欧氏范数就是其**长度**。

L2范数：向量所有元素的平方和的开平方。

两个向量完全指向相同的方向时，余弦相似度为1；完全指向相反的方向时，余弦相似度为-1。

In [ ]:
def cos_similarity(x, y, eps=1e-8):
    # x, y 均为向量, eps 微小值解决向量为0时的除数为0的问题,绝大多数情况不会对结果影响
    nx = x / np.sqrt(np.sum(x ** 2) + eps)  # x ** 2 是两个向量x相乘，即向量内积，各元素的平方和， sqrt 对平方和 开平方
    ny = y / np.sqrt(np.sum(y ** 2) + eps)
    return np.dot(nx, ny)


# you 和 i 的相似度
c0 = C[word_to_id['you']]  # you的向量
c1 = C[word_to_id['i']]  # i的向量
print(cos_similarity(c0, c1))

## 相似单词的排序
将某个单词的相似词按降序展示

In [ ]:
def most_similar(query, word_to_id, id_to_word, co_matrix, top=5):
    # 取出查询词
    if query not in word_to_id:
        print(query, 'is not found')
        return

    query_id = word_to_id[query]
    query_vec = co_matrix[query_id]

    # 计算相似度
    vocab_size = len(word_to_id)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(co_matrix[i], query_vec)

    # 按相似度降序输出 argsort 对向量升序排序并返回原索引，-1 则降序
    count = 0
    for i in (-1 * similarity).argsort():
        if i == query_id:
            continue
        print(id_to_word[i], similarity[i])
        count += 1
        if count >= top:
            return


most_similar('you', word_to_id, id_to_word, C)

goodbye 和 you 这么相似，明显不符合预期。